I strongly recommend to go through the article [here](https://www.analyticsvidhya.com/blog/2019/07/learn-build-first-speech-to-text-model-python/) to understand the basics of signal processing prior implementing the speech to text.

**Understanding the Problem Statement for our Speech-to-Text Project**

Let’s understand the problem statement of our project before we move into the implementation part.

We might be on the verge of having too many screens around us. It seems like every day, new versions of common objects are “re-invented” with built-in wifi and bright touchscreens. A promising antidote to our screen addiction is voice interfaces. 

TensorFlow recently released the Speech Commands Datasets. It includes 65,000 one-second long utterances of 30 short words, by thousands of different people. We’ll build a speech recognition system that understands simple spoken commands.

You can download the dataset from [here](https://www.kaggle.com/c/tensorflow-speech-recognition-challenge).

**Implementing the Speech-to-Text Model in Python**

The wait is over! It’s time to build our own Speech-to-Text model from scratch.

**Import the libraries**

First, import all the necessary libraries into our notebook. LibROSA and SciPy are the Python libraries used for processing audio signals.

In [2]:
import os
import librosa
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import wavfile
import warnings
import glob

warnings.filterwarnings("ignore")

**Data Exploration and Visualization**

Data Exploration and Visualization helps us to understand the data as well as pre-processing steps in a better way. 

**Visualization of Audio signal in time series domain**

Now, we’ll visualize the audio signal in the time series domain:

In [2]:
train_audio_path = '/Users/vsatpathy/Desktop/Python/train/'
samples, sample_rate = librosa.load('bed.wav', sr = 16000)
fig = plt.figure(figsize=(14, 8))
ax1 = fig.add_subplot(211)
ax1.set_title('Raw wave of ' + 'bed.wav')
ax1.set_xlabel('time')
ax1.set_ylabel('Amplitude')
ax1.plot(np.linspace(0, sample_rate/len(samples), sample_rate), samples)

**Sampling rate **

Let us now look at the sampling rate of the audio signals

In [3]:
ipd.Audio(samples, rate=sample_rate)

In [4]:
print(sample_rate)

16000


**Resampling**

From the above, we can understand that the sampling rate of the signal is 16000 hz. Let us resample it to 8000 hz since most of the speech related frequencies are present in 8000z 

In [5]:
samples = librosa.resample(samples, sample_rate, 8000)
ipd.Audio(samples, rate=8000)

Now, let’s understand the number of recordings for each voice command:

In [6]:
labels=os.listdir(train_audio_path)
#labels.pop(0)
fin=glob.glob(train_audio_path+'*'+'.wav')
fin

['/Users/vsatpathy/Desktop/Python/train/open_1.wav',
 '/Users/vsatpathy/Desktop/Python/train/close_4.wav',
 '/Users/vsatpathy/Desktop/Python/train/close_5.wav',
 '/Users/vsatpathy/Desktop/Python/train/open_0.wav',
 '/Users/vsatpathy/Desktop/Python/train/open_2.wav',
 '/Users/vsatpathy/Desktop/Python/train/close_7.wav',
 '/Users/vsatpathy/Desktop/Python/train/close_6.wav',
 '/Users/vsatpathy/Desktop/Python/train/open_3.wav',
 '/Users/vsatpathy/Desktop/Python/train/open_7.wav',
 '/Users/vsatpathy/Desktop/Python/train/close_2.wav',
 '/Users/vsatpathy/Desktop/Python/train/close_3.wav',
 '/Users/vsatpathy/Desktop/Python/train/open_6.wav',
 '/Users/vsatpathy/Desktop/Python/train/open_4.wav',
 '/Users/vsatpathy/Desktop/Python/train/no_8.wav',
 '/Users/vsatpathy/Desktop/Python/train/close_1.wav',
 '/Users/vsatpathy/Desktop/Python/train/close_0.wav',
 '/Users/vsatpathy/Desktop/Python/train/no_9.wav',
 '/Users/vsatpathy/Desktop/Python/train/open_5.wav',
 '/Users/vsatpathy/Desktop/Python/train/st

In [10]:
labels=["close","no","open","run","stop","yes"]

**Preprocessing the audio waves**

In the data exploration part earlier, we have seen that the duration of a few recordings is less than 1 second and the sampling rate is too high. So, let us read the audio waves and use the below-preprocessing steps to deal with this.

Here are the two steps we’ll follow:

* Resampling
* Removing shorter commands of less than 1 second

Let us define these preprocessing steps in the below code snippet:

In [11]:
train_audio_path = '/Users/vsatpathy/Desktop/Python/train/'

all_wave = []
all_label = []

waves = [f for f in os.listdir(train_audio_path) if f.endswith('.wav')]
for wav in waves:
    #print(wav)
    samples, sample_rate = librosa.load(train_audio_path+wav, sr = 16000)
    samples = librosa.resample(samples, sample_rate, 16000)
    if(len(samples)== 16000) : 
        all_wave.append(samples)
        all_label.append(wav.split('_')[0])
'''
for label in labels:
    #print(label)
    for i in range(0,int(len(all_wave)/len(labels))):
        #all_label.append(label)
'''
all_label[1]

'close'

Convert the output labels to integer encoded:

In [12]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y=le.fit_transform(all_label)
classes= list(le.classes_)
classes

['close', 'no', 'open', 'run', 'stop', 'yes']

Now, convert the integer encoded labels to a one-hot vector since it is a multi-classification problem:

In [13]:
from keras.utils import np_utils
y=np_utils.to_categorical(y, num_classes=len(labels))
y.shape

(60, 6)

Reshape the 2D array to 3D since the input to the conv1d must be a 3D array:

In [14]:
all_wave = np.array(all_wave).reshape(-1,16000,1)
all_wave.shape

(60, 16000, 1)

**Split into train and validation set**

Next, we will train the model on 80% of the data and validate on the remaining 20%:


In [15]:
from sklearn.model_selection import train_test_split
x_tr, x_val, y_tr, y_val = train_test_split(np.array(all_wave),np.array(y),stratify=y,test_size = 0.15,random_state=777,shuffle=True)

**Model Architecture for this problem**

We will build the speech-to-text model using conv1d. Conv1d is a convolutional neural network which performs the convolution along only one dimension. 

**Model building**

Let us implement the model using Keras functional API.

In [2]:
from keras.layers import Dense, Dropout, Flatten, Conv1D, Input, MaxPooling1D
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
K.clear_session()

inputs = Input(shape=(16000,1))

#First Conv1D layer
conv = Conv1D(8,13, padding='valid', activation='relu', strides=1)(inputs)
conv = MaxPooling1D(3)(conv)
conv = Dropout(0.3)(conv)

#Second Conv1D layer
conv = Conv1D(16, 11, padding='valid', activation='relu', strides=1)(conv)
conv = MaxPooling1D(3)(conv)
conv = Dropout(0.3)(conv)

#Third Conv1D layer
conv = Conv1D(32, 9, padding='valid', activation='relu', strides=1)(conv)
conv = MaxPooling1D(3)(conv)
conv = Dropout(0.3)(conv)

#Fourth Conv1D layer
conv = Conv1D(64, 7, padding='valid', activation='relu', strides=1)(conv)
conv = MaxPooling1D(3)(conv)
conv = Dropout(0.3)(conv)

#Flatten layer
conv = Flatten()(conv)

#Dense Layer 1
conv = Dense(256, activation='relu')(conv)
conv = Dropout(0.3)(conv)

#Dense Layer 2
conv = Dense(128, activation='relu')(conv)
conv = Dropout(0.3)(conv)

outputs = Dense(6, activation='softmax')(conv)

model = Model(inputs, outputs)
model.summary()

Using TensorFlow backend.



Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 16000, 1)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 15988, 8)          112       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 5329, 8)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 5329, 8)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 5319, 16)          1424      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 1773, 16)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 1773, 16)          0  

Define the loss function to be categorical cross-entropy since it is a multi-classification problem:

In [14]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [15]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, min_delta=0.0001) 
mc = ModelCheckpoint('best_model_2.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')

In [16]:
history=model.fit(x_tr, y_tr ,epochs=100, batch_size=8, validation_data=(x_val,y_val),callbacks=[es,mc])
model.save('best_model_2.h5')


Train on 51 samples, validate on 9 samples
Epoch 1/100
51/51 [==============================] - 1s 17ms/step - loss: 1.8644 - accuracy: 0.2157 - val_loss: 1.7419 - val_accuracy: 0.3333
Epoch 2/100
51/51 [==============================] - 1s 10ms/step - loss: 1.7407 - accuracy: 0.1961 - val_loss: 1.7385 - val_accuracy: 0.4444
Epoch 3/100
51/51 [==============================] - 1s 10ms/step - loss: 1.6376 - accuracy: 0.3137 - val_loss: 1.7091 - val_accuracy: 0.4444
Epoch 4/100
51/51 [==============================] - 1s 11ms/step - loss: 1.6048 - accuracy: 0.3529 - val_loss: 1.6061 - val_accuracy: 0.5556
Epoch 5/100
51/51 [==============================] - 1s 10ms/step - loss: 1.5566 - accuracy: 0.2941 - val_loss: 1.5690 - val_accuracy: 0.5556
Epoch 6/100
51/51 [==============================] - 1s 10ms/step - loss: 1.3894 - accuracy: 0.4118 - val_loss: 1.4685 - val_accuracy: 0.6667
Epoch 7/100
51/51 [==============================] - 1s 10ms/step - loss: 1.3607 - accuracy: 0.5490 - va

**Diagnostic plot**

I’m going to lean on visualization again to understand the performance of the model over a period of time:

In [17]:
from matplotlib import pyplot
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

**Loading the best model**

In [16]:
from keras.models import load_model
model=load_model('six_label_model.h5')

Define the function that predicts text for the given audio:

In [17]:
def predict(audio):
    prob=model.predict(audio.reshape(1,16000,1))
    index=np.argmax(prob[0])
    return classes[index]

Prediction time! Make predictions on the validation data:

In [18]:
import random
index=random.randint(0,len(x_val)-1)
samples=x_val[index].ravel()
print("Audio:",classes[np.argmax(y_val[index])])
ipd.Audio(samples, rate=16000)

Audio: open


In [19]:
print("Text:",predict(samples))

Text: open
